In [ ]:
%%bash
#pip install TextBlob
#conda install gensim

In [35]:
import pandas as pd
import numpy as np
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
from gensim import corpora, models, similarities
import slugify as sl

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"
movieplots = pd.read_csv(dropbox + 'movieplots.csv')
#this is a work around because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

In [3]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [4]:
movieplots = movieplots.apply(clean, axis = 1)
dftouse = movieplots[movieplots.year > 2014]

In [5]:
print movieplots.shape
print dftouse.shape

(259028, 5)
(16943, 5)


In [6]:
import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)   

In [7]:
plots_sdf = sqlsc.createDataFrame(dftouse[:100])

In [45]:
def slugify (text):
    return sl.slugify(text)

In [8]:
def sentences(text):
    blob = tb.TextBlob(text)
    return blob.sentences

In [9]:
def stem(word):
    return PorterStemmer().stem(word)

In [12]:
num = re.compile('\d')
def replace(word):
    return bool(num.search(word))

In [47]:
def process(sents, dostop=True, dostem=True, donum=True, do=True):
    stopwords = set(sw.words('english'))
    sents_processed  = []
    words_processed  = []
    for sent in sents:
        words = []
        for word in sent.words:
            if dostop:
                if word in stopwords:
                    continue
            if donum:
                if contains_number(word):
                    continue
            if douni:
                word = slugify(word)
            if dostem:
                word = stem(word)
            words.append(word.lower())
        sents_processed.append(words)
        words_processed += words 
    return (sents_processed, words_processed)

In [48]:
plots = (plots_sdf[['title','plots']]
    .map(lambda x : (' '.join(x[1])))
)
docs = (plots
    .map(sentences)
    .map(process)          
).cache()
docs_sents = docs.map(lambda x : x[0]).collect()
docs_words = docs.map(lambda x : x[1]).collect()

In [50]:
dictionary = corpora.Dictionary(docs_words)
#dictionary.save(dropbox +"testdict.dict") 
dictionary.token2id

2394

In [ ]:
corpus = (dictionary.doc2bow(doc) for doc in docs_words)
#corpora.MmCorpus.serialize(dropbox +"testcorpus.mm", corpus)

In [ ]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [44]:
plot=[u'Jules is, self declared, the most useless person in the post apocalyptic world, until he finds an old film camera and determines to make the greatest movie in the new world... the only movie in the new world. But his first day filming is proving to be much more difficult than he imagined. Who knew that making a movie after the end of the world would be so hard?']
slugify(sentences(plot[0])[0].words[2])

'self.'

In [ ]:
sdf